
Going to predict the spreads for games on march 17th


In [ ]:

#import sys
#import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
#stderr = sys.stderr
#sys.stderr = open(os.devnull, 'w')

import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  
from sklearn import preprocessing, cross_validation
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense
import tensorflow 

In [8]:
def make_network(FILENAME):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    from sklearn.neural_network import MLPClassifier
    """Given the csv input of all the box scores, arrange it such that the home and away teams are lined up, 
    unnecessary columns removed, and proper hot encoding is done. Other stuff too probably.
    
    Note that this data has already been doctored from its original form, taking out most unnecessary columns but
    those could be useful later on.
    
    
    Parameters
    ----------
    FILENAME : file
        The csv of the data.
        
    Returns
    -------
    
    X_train : array
        Numpy array of the training inputs.
    y_train : array
        Numpy array of the training outputs.
    
    X_test : array
        Numpy array of the testing inputs.
    y_test : array
        Numpy array of the testing outputs.
    Sike!
    
    model : object
        MLP which can predict the outcome of NBA games
    """
    #Read in file, remove attempted and # and only account for % since that's more predictive in nature. 
    data = read_csv(FILENAME)
    data['3P%'] = np.divide(data['3P'].values,data['3PA'].values)
    del data['3P'],data['3PA']
    data['FG%'] = np.divide(data['FG'].values,data['FGA'].values)
    del data['FG'],data['FGA']
    data['FT%'] = np.divide(data['FT'].values,data['FTA'].values)
    del data['FT'],data['FTA']
    data = get_dummies(data)
    del data['VENUE_Home'],data['VENUE_Road']
    #print(data)
    
    

    dat = []

    for i in range(len(data.values)):
        data.values[i] = np.reshape(data.values[i],newshape=[1,len(data.values[i])])
    for p in range(int(len(data.values)/2)):
        fullboxgame = np.concatenate((data.values[2*p],data.values[(2*p)+1]))
        dat.append(fullboxgame)
    

    """At this point in the data dat is an array arranged as 
        OR  DR  TOT   A  PF  ST  TO  BL  PTS       3P%       FG%       FT% x2  (for road then home )
    so next up is to hot pull road and home points 
    
    road points is column 8 and home points is column 20. 
    
    So grab those from dat and make y. 
    
    X is every other column, so merge everyone else 
    
    """
    dat = np.array(dat)   
    roadpts = dat[:,8]

    homepts = dat[:,21]
    endspreadS = roadpts-homepts
    openingspreadS = dat[:,9]
#    print("opening spread:" + str(dat[:,9]))
    y = []
    for j in range(len(endspreadS)):
        openspread = openingspreadS[j]
       # print("this is the spread of the road team " + str(openspread))
        endspread = endspreadS[j]
       # print("the road team won by  .. " + str(endspread))
       # if endspread>openspread:
        #    y.append(np.array([0,1,0]))  #OK, now make sure this is formateed properly!
        if openspread + endspread <0:
            y.append(np.array([0,1,0]))
        elif openspread + endspread >0:
            y.append(np.array([1,0,0]))
        else: 
            y.append(np.array([0,0,1]))



    y = np.array(y)
    X1 = np.concatenate((dat[:,0:8],dat[:,10:21]),axis=1)
    X = np.concatenate((X1,dat[:,23:26]),axis=1)    #need to go one further column to snag HFT% 
    X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.33)
    #print((X[0]))
    #print(np.shape(X[0]))
    INPUT_DIM = len(X[0])
    model = Sequential()
    model.add(Dense(30,input_dim=INPUT_DIM,activation='relu'))
    model.add(Dense(30,input_dim=INPUT_DIM,activation='relu'))


    model.add(Dense(3,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.fit(X_train,y_train,batch_size=32,epochs=20,validation_split=.2,verbose=1,)

    print(model.evaluate(X_test,y_test))
    return model
    

If spread of road team is + that means they are supposed to lose by that much
if the end spread is - that means the road team lost by that much


In [9]:
noahbets = make_network('1517_boxscores.csv')

Train on 1406 samples, validate on 352 samples
Epoch 1/20
1406/1406 [==============================] - 0s 351us/step - loss: 2.5544 - acc: 0.5263 - val_loss: 0.9665 - val_acc: 0.5938
Epoch 2/20
1406/1406 [==============================] - 0s 49us/step - loss: 0.9286 - acc: 0.6430 - val_loss: 0.7588 - val_acc: 0.6619
Epoch 3/20
1406/1406 [==============================] - 0s 45us/step - loss: 0.7844 - acc: 0.6885 - val_loss: 0.7104 - val_acc: 0.6591
Epoch 4/20
1406/1406 [==============================] - 0s 46us/step - loss: 0.7049 - acc: 0.6984 - val_loss: 0.6505 - val_acc: 0.6989
Epoch 5/20
1406/1406 [==============================] - 0s 46us/step - loss: 0.6496 - acc: 0.7319 - val_loss: 0.6121 - val_acc: 0.7301
Epoch 6/20
1406/1406 [==============================] - 0s 47us/step - loss: 0.6203 - acc: 0.7376 - val_loss: 0.5864 - val_acc: 0.7500
Epoch 7/20
1406/1406 [==============================] - 0s 46us/step - loss: 0.5944 - acc: 0.7589 - val_loss: 0.6026 - val_acc: 0.7415
Epoch 8

In [10]:
def make_prediction_data(filename):
    from pandas import read_csv,get_dummies
    import numpy as xnp
    from sklearn import cross_validation
    data = read_csv(filename)
    #convert to per game stats and sort columns. 
    data['ORB'] =  np.divide(data['ORB'].values,data['G'].values)
    data['DRB'] =  np.divide(data['DRB'].values,data['G'].values)
    data['TRB'] =  np.divide(data['TRB'].values,data['G'].values)
    data['AST'] =  np.divide(data['AST'].values,data['G'].values)
    data['STL'] =  np.divide(data['STL'].values,data['G'].values)
    data['BLK'] =  np.divide(data['BLK'].values,data['G'].values)
    data['TOV'] =  np.divide(data['TOV'].values,data['G'].values)
    data['PF'] =  np.divide(data['PF'].values,data['G'].values)
    teams  = data['Team']

    data = data[['ORB' , 'DRB'  ,'TRB' ,  'AST' , 'PF' , 'STL' , 'TOV' , 'BLK' ,'3P%','FG%' ,'FT%']]
    print("Here is every teams index value: ")
    print(teams)
    return teams,data

In [11]:
def game_maker(roadteam,hometeam):
    """After creating a properly formated table, this concats the desired teams so they can be predicted. 
        Based on get team index # based on output of predictor, and make it the input for stats ie GSW are stats[0].
        and so on!
    """
    game = np.concatenate((roadteam,hometeam))
    return game

In [12]:
teams, stats =make_prediction_data('teamstats_317.csv')

Here is every teams index value: 
0     Golden State Warriors*
1     Toronto Raptors*      
2     Houston Rockets*      
3     New Orleans Pelicans  
4     Oklahoma City Thunder 
5     Minnesota Timberwolves
6     Denver Nuggets        
7     Cleveland Cavaliers   
8     Los Angeles Lakers    
9     Los Angeles Clippers  
10    Washington Wizards    
11    Charlotte Hornets     
12    Philadelphia 76ers    
13    Orlando Magic         
14    Brooklyn Nets         
15    Phoenix Suns          
16    Indiana Pacers        
17    Boston Celtics*       
18    Portland Trail Blazers
19    New York Knicks       
20    Miami Heat            
21    Atlanta Hawks         
22    Milwaukee Bucks       
23    Utah Jazz             
24    Dallas Mavericks      
25    San Antonio Spurs     
26    Chicago Bulls         
27    Detroit Pistons       
28    Sacramento Kings      
29    Memphis Grizzlies     
30    League Average        
Name: Team, dtype: object


In [13]:
stats.values[1]

array([ 9.97101449, 34.04347826, 44.01449275, 23.91304348, 21.85507246,
        8.01449275, 13.24637681,  6.04347826,  0.356     ,  0.47      ,
        0.803     ])

In [14]:
game1 = game_maker(stats.values[24],stats.values[14])
game2 = game_maker(stats.values[27],stats.values[18])
game3 = game_maker(stats.values[21],stats.values[22])
game4 =  game_maker(stats.values[16],stats.values[10])
game5 = game_maker(stats.values[2],stats.values[3])
game6 = game_maker(stats.values[11],stats.values[19])
game7 = game_maker(stats.values[7],stats.values[26])
game8 = game_maker(stats.values[6],stats.values[29])
game9 = game_maker(stats.values[5],stats.values[25])
game10 =game_maker(stats.values[28],stats.values[23])
game11 =game_maker(stats.values[0],stats.values[15])

In [15]:
noahbets.predict(np.array([game1,game2,game3,game4,game5,game6,game7,game8,game9,game10,game11]))

array([[0.7600634 , 0.2301049 , 0.00983162],
       [0.64130557, 0.34647855, 0.01221585],
       [0.69364244, 0.29266167, 0.01369589],
       [0.7222185 , 0.26769456, 0.01008693],
       [0.6267847 , 0.3654825 , 0.00773278],
       [0.84223765, 0.15161434, 0.00614797],
       [0.5907948 , 0.39573064, 0.01347451],
       [0.85058016, 0.14037517, 0.0090446 ],
       [0.6027452 , 0.38778976, 0.00946503],
       [0.54237974, 0.44373718, 0.01388317],
       [0.9197156 , 0.07308447, 0.00720002]], dtype=float32)

Winners 3/10:
10 Hornets Y
24 Heat Y
27 Spurs N
26 Mavs Y
12 Clippers Y

Winners 3/11:
4 Raptors Y
25 Bulls Y
0 GSW N
3 Pelicans N
5 Nuggets Y
26 Rockets Y
13 Nets N
7 Cavs N
18 Celtics N

Winners 3/12:
Blazers Y
Spurs N
Bucks Y
Thunder Y


Winners 3/13:

Philly N
Wash N
OKC Y
Toronto Y
Dallas Y
Bulls N
Pels Y
Spurs Y
UTAH Y
Cle Y
Lakers Y

20-9

Predicted winners on March 9th:

Indiana
Chicago 
toronto
nop
utah
knicks    *
lakers
orlando  *
warriors
clippers


In [ ]:
20/29

Arrangement of Data:
Away :      OR  DR  TOT   A  PF  ST  TO  BL  PTS   3P%       FG%       FT%
  Home: OR  DR  TOT   A  PF  ST  TO  BL  3P%  FG%  FT%| AW HW
      0    1   2   3   4   5  6    7   8    9   10  

Data:
1. Road Orbs
2. Road Drbs
3. Road Trbs
4. Road Asts
5. Road Pfs
6. Road Stls
7. Road Tos. 
8. Road Blks 
9. Road 3p%
10. Road Fg%
11. Road Ft% 
12. Road venue Home (0)  REMOVED
13. Road venue Away (1) REMOVED
14. Home Orbs
15. Home Drbs
16. Home Trbs
17. Home Asts
18. Home Pfs
19. Home Stls
20. Home Tos. 
21. Home Blks 
22. Home 3p%
23. Home Fg%
24. Home Ft% 
25. Home venue Home (0) REMOVED
26. Home venue Away (1) REMOVED

With Outputs 

27. Home team win (1)
28. Road team win (0)


So you enter the statistics of both teams, and it gives a classification of whether or not the home or away team wins.
Unclear how good this actually is at characterizing teams. Should use team stats from as recent a metric as possible to reflect current gameplay


Let's do a raw version where I pull the last 5 stats of wiz vs pels 

3/8 Games and if it was correct:

Suns @ OKC YES

Nets @ HOR NO

Bos @ Min NO

Phi @ Mia YES

Sas @ GSW Says spurs, 

This is a good start. What's left?

1. What about assists, rebounds, steals, etc.? Can a table do that too? 

2. Hot encode wins and losses. Need to move that result column and delete everyhting except W and L.

3. Normalize data?

4. Create a net

5. Figure out prediction data, go by season averages? L5 averages? 

Remove unneccesary stats. Flatten 2 adjascent rows because they're the same game!